# Elastic Net Regression

In [1]:
# extension that autoformats all code to PEP8
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go

C:\Users\Gianni\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


<IPython.core.display.Javascript object>

The advantage of Elastic Net is that we get the best of both worlds: the dimnesion reduction by shrinkage without ignoring highly correlated variables in groups.Our goal in Elastic Net is to minimize the following loss function:
$$
1 / (2 * n_{samples}) * ||y - Xw||^2_2
+ alpha * l_1 ratio * ||w||_1
+ 0.5 * alpha * (1 - l_1 ratio) * ||w||^2_2
$$ 
or 
$$
a * ||w||_1 + 0.5 * b * ||w||_2^2
$$

## Data Loading and Cleaning

In [22]:
# Using updated dataframe from preprocessing notebook
df = pd.read_csv("../data/listings.csv")
# Random column of index
df = df.drop(["Unnamed: 0"], axis=1,)
display(df)

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
0,149,1,9,0.21,6,365,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1235
1,225,1,45,0.38,2,355,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1021
2,89,1,270,4.64,1,194,0,1,0,0,...,0,0,0,0,0,0,1,0,0,976
3,80,10,9,0.10,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1204
4,200,3,74,0.59,1,129,0,0,1,0,...,0,0,0,0,0,0,1,0,0,989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,129,1,1,1.00,1,147,0,0,1,0,...,0,0,0,0,0,0,0,1,0,974
38833,45,1,1,1.00,6,339,0,0,0,1,...,0,0,0,0,0,0,0,1,0,974
38834,235,1,1,1.00,1,87,0,0,0,0,...,0,0,0,0,0,0,0,1,0,974
38835,100,1,2,2.00,1,40,1,0,0,0,...,0,0,0,0,0,0,1,0,0,974


<IPython.core.display.Javascript object>

In [38]:
# Who needs a black box?
def standardizeData(data):
    # Takes pandas dataframe and standardizes it iteratively
    for column in data:
        if (data[column] != 0).sum() == 0:  # If the column is all zeros
            continue  # No need to standardize all zeros (thus dividing by zero)
        else:
            data[column] = (data[column] - np.mean(data[column])) / np.std(data[column])
    return data

<IPython.core.display.Javascript object>

In [40]:
# Create training data, holding out 10% of data for testing
from sklearn.model_selection import train_test_split  # Returns pandas dataframe

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    df.drop("price", axis=1), df["price"], test_size=0.2, random_state=42
)
# display(Xtrain["neighbourhood_Willowbrook"])
# # Standardize data for faster convergence in gradient descent
display(Xtest.isnull().values.any())
print((Xtest["neighbourhood_Willowbrook"] != 0).sum())
Xtrain, Xtest = standardizeData(Xtrain), standardizeData(Xtest)

# Standardize data for faster convergence in gradient descent
# from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# Xtrain, Xtest = sc.fit_transform(Xtrain), sc.fit_transform(Xtest)

print(Xtrain.isnull().values.any())
display(Xtrain)
display(Xtest.isnull().values.any())
display(Xtest)

False

0
False


,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
6263,-0.167611,2.561530,0.915898,-0.158841,0.947496,-0.152154,-0.860558,1.159744,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,-1.045753,1.092554,-0.14925,-0.640362
18571,-0.226746,0.967890,0.844265,-0.120813,1.194061,-0.152154,1.162037,-0.862259,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.637946
31589,-0.167611,-0.563660,-0.588380,-0.158841,-0.886333,-0.152154,-0.860558,1.159744,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.570287
25228,-0.285880,0.119329,0.486104,-0.158841,1.856705,-0.152154,-0.860558,-0.862259,2.739112,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.652444
32717,-0.285880,-0.025548,1.912780,-0.158841,-0.031060,-0.152154,-0.860558,1.159744,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,-1.045753,1.092554,-0.14925,-0.657277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,-0.108477,1.857845,0.515951,-0.158841,1.217177,-0.152154,1.162037,-0.862259,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.628280
11284,-0.167611,-0.460177,-0.689859,0.297494,-0.316151,-0.152154,1.162037,-0.862259,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.521958
38158,-0.285880,-0.501570,2.169462,-0.158841,0.069107,-0.152154,-0.860558,1.159744,-0.365082,-0.08897,...,-0.297550,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.642779
860,-0.167611,-0.087637,-0.642104,-0.158841,-0.886333,-0.152154,1.162037,-0.862259,-0.365082,-0.08897,...,3.360776,-0.005673,-0.057109,-0.042112,-0.017022,-0.065071,0.956249,-0.915287,-0.14925,-0.193328


False

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
8022,-0.208915,-0.242654,-0.608102,-0.156562,1.289191,-0.150924,1.186565,-0.877428,-0.366532,-0.095359,...,3.348707,0,-0.059059,-0.046832,-0.016048,-0.071035,-1.056703,1.103990,-0.14913,-0.539198
3875,-0.000242,-0.599271,-0.813864,-0.156562,-0.889135,-0.150924,-0.842769,1.139695,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,0.946340,-0.905805,-0.14913,3.931065
18829,-0.261084,-0.305586,-0.461129,-0.156562,-0.889135,-0.150924,-0.842769,1.139695,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,0.946340,-0.905805,-0.14913,0.394526
32406,-0.208915,-0.347541,0.250219,-0.118540,-0.134501,6.625838,-0.842769,-0.877428,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,0.946340,-0.905805,-0.14913,-0.609348
26597,1.251799,-0.578293,-0.678649,0.033548,1.094697,-0.150924,1.186565,-0.877428,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,0.946340,-0.905805,-0.14913,-0.161838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,-0.052410,1.057951,-0.143668,-0.118540,1.429226,-0.150924,-0.842769,-0.877428,2.728274,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,-1.056703,1.103990,-0.14913,-0.585159
14280,-0.208915,0.302761,-0.043726,-0.156562,-0.881356,-0.150924,1.186565,-0.877428,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,-1.056703,1.103990,-0.14913,-0.638376
11098,-0.261084,-0.599271,-0.813864,-0.156562,-0.889135,-0.150924,-0.842769,1.139695,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,-1.056703,1.103990,-0.14913,2.462748
17025,-0.261084,-0.536338,-0.743317,-0.156562,-0.889135,-0.150924,-0.842769,1.139695,-0.366532,-0.095359,...,-0.298623,0,-0.059059,-0.046832,-0.016048,-0.071035,0.946340,-0.905805,-0.14913,1.383886


<IPython.core.display.Javascript object>

## Performing Elastic Net Regression

In [44]:
# doing Elastic Net by hand via a class
# https://math.stackexchange.com/questions/1111504/differentiation-with-respect-to-a-matrix-residual-sum-of-squares
class ElasticRegression:
    def __init__(self, learning_rate, iterations, l1_penalty, l2_penalty):

        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l1_penalty = l1_penalty
        self.l2_penalty = l2_penalty

    # Function for model training
    def fit(self, X, Y):

        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros(self.n)
        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning
        for i in range(self.iterations):
            self.update_weights()

        return self

    # Helper function to update weights in gradient descent
    def update_weights(self):
        Y_pred = self.predict(self.X)

        # calculate gradients
        dW = np.zeros(self.n)

        for j in range(self.n):
            if self.W[j] > 0:
                dW[j] = (
                    -(2 * np.transpose(self.X[:, j]) @ (self.Y - Y_pred))
                    + self.l1_penalty
                    + 2 * self.l2_penalty * self.W[j]
                ) / self.m
            else:
                dW[j] = (
                    -(2 * np.transpose(self.X[:, j]) @ (self.Y - Y_pred))
                    - self.l1_penalty
                    + 2 * self.l2_penalty * self.W[j]
                ) / self.m
        db = -2 * np.sum(self.Y - Y_pred) / self.m

        # update weights
        self.W = self.W - self.learning_rate * dW

        self.b = self.b - self.learning_rate * db

        return self

    # Hypothetical function  h( x )
    def predict(self, X):
        return X @ self.W + self.b

<IPython.core.display.Javascript object>

### Testing with the 'Iris' Data Set

In [45]:
# Load in iris from seaborn library
iris = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
)
iris = iris.drop("species", axis=1)
display(iris)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


<IPython.core.display.Javascript object>

In [46]:
Xtrain_iris, Xtest_iris, Ytrain_iris, Ytest_iris = train_test_split(
    iris.drop("sepal_length", axis=1),
    iris["sepal_length"],
    test_size=0.2,
    random_state=42,
)

# Standardize our X matrices
Xtrain_iris, Xtest_iris = standardizeData(Xtrain_iris), standardizeData(Xtest_iris)

# Do the model
model = ElasticRegression(
    iterations=100, learning_rate=0.01, l1_penalty=0.99, l2_penalty=0.01
)
model.fit(Xtrain_iris.to_numpy(), Ytrain_iris.to_numpy())

Y_pred_iris = model.predict(Xtest_iris)
display(pd.DataFrame({"Predictions": Y_pred_iris, "Actual": Ytest_iris}).head())

,Predictions,Actual
73,5.112736,6.1
18,4.446238,5.7
118,5.964557,7.7
78,5.219452,6.0
76,5.212697,6.8


<IPython.core.display.Javascript object>

### On to Our AirBnB Data

In [52]:
enm = ElasticRegression(
    iterations=10000, learning_rate=0.01, l1_penalty=0.99, l2_penalty=0.01
)

# display(Xtrain)
enm.fit(Xtrain.to_numpy(), Ytrain.to_numpy())

Ypred = enm.predict(Xtest)

display(pd.DataFrame({"Predictions": np.round(Ypred, 2), "Actual": Ytest}))

# print("Trained W: ", round(enm.W[0], 2))
# print("Trained b: ", round(enm.b, 2))

,Predictions,Actual
8022,126.45,90
3875,195.59,150
18829,175.80,145
32406,131.20,120
26597,181.22,62
...,...,...
2923,89.54,56
14280,26.57,60
11098,123.54,60
17025,212.21,97


<IPython.core.display.Javascript object>

In [10]:
### WARNING: BE CAREFUL WHEN RUNNING THIS, IT WILL USE YOUR ENTIRE CPU!!!

# # Performing Elastic Net with sklearn and CV
# import time
# from sklearn.linear_model import ElasticNetCV
# from sklearn.model_selection import RepeatedKFold
# import warnings

# # Muting the 1000 warnings this cell will output
# warnings.filterwarnings("ignore")

# # Measure run time
# start = time.time()

# # Method of model eval
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# # Defining lists for model eval
# ratios = np.arange(0, 1, 0.01)
# alphas = np.arange(
#     0, 1, 0.01
# )  # [0.0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5, 0.75, 1.0, 10.0, 100.0]


# enm = ElasticNetCV(
#     l1_ratio=ratios, alphas=alphas, fit_intercept=False, cv=cv, n_jobs=-1
# )  # n_jobs = -1 will use all processors on CPU
# enm.fit(Xtrain, Ytrain)


# print("--- %s seconds ---" % (time.time() - start))

<IPython.core.display.Javascript object>

In [11]:
# Lets take the coefficients of training and the names of our columns and put them into a dataframe
variables = Xtrain.columns.tolist()
model_coef = pd.DataFrame({"coef": enm.coef_, "names": variables})

print("alpha: %f" % enm.alpha_)
print("l1_ratio_: %f" % enm.l1_ratio_)

# show all the coef that are non zero in descending order
display(model_coef[model_coef["coef"] != 0].sort_values(by="coef", ascending=False))


# print(enm.mse_path_[0][0])
enm.alphas_

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

<IPython.core.display.Javascript object>

In [ ]:
# These next three cells will be dedicated to plotting values used in CV
MSE = []
l1col = np.concatenate([np.repeat(x, 100) for x in np.arange(0, 1, 0.01)])
alphacol = np.flip(
    np.concatenate([np.arange(0, 1, 0.01) for x in np.arange(0, 1, 0.01)])
)
for i in range(len(enm.mse_path_)):
    for j in range(len(enm.mse_path_[i])):
        MSE.append(enm.mse_path_[i][j])

In [ ]:
kFolds = pd.DataFrame(np.array(MSE))
minkFolds = pd.DataFrame(kFolds.min(axis=1), columns=["min"])
graphdf = pd.DataFrame({"l1": l1col, "alpha": alphacol})
graphdf = graphdf.join(minkFolds)
# display(graphdf)

In [ ]:
# Plotting 3 dimensional plot
fig = go.Figure(
    data=px.scatter_3d(
        graphdf,
        x="l1",
        y="alpha",
        z="min",
        opacity=0.05,
        labels={
            "l1": "LASSO parameter",
            "alpha": "Ridge parameter",
            "min": "Minimumn MSE",
        },
        color_discrete_sequence=px.colors.qualitative.Safe,
        title="Minimum MSE in 3 Trial 10-Fold CV with Combined LASSO and Ridge Penalty Values",
    )
)
fig.show()

In [ ]:
# Now lets predict with our testing data
predictions = enm.predict(Xtest)

# Compared to the actual values
pred_v_act = pd.DataFrame({"predictions": enm.predict(Xtest), "actual": Ytest})
display(pred_v_act)

from sklearn.metrics import *

# Metrics
score = r2_score(Ytest, predictions)
MAE = mean_absolute_error(Ytest, predictions)
MSE = mean_squared_error(Ytest, predictions)

# Printing of Metrics
print("The r-squared value is ", score)
print(
    "The adjusted r-squared value is ",
    1
    - (
        ((1 - score) * (len(pred_v_act.index) - 1))
        / (len(pred_v_act) - len(pred_v_act.columns) - 1)
    ),
)
print("The Mean Absolute Error is ", MAE)
print("The root Mean Squared Error is ", np.sqrt(MSE))